# Piecewise task

Email:

```
I have a piecewise linear obj function in a blend composition problem. Say, x1, x2 .. x7 (continuous, lbound = 0 and ubound = 1) are my decision variable (composition of different material grades) where x1+x2+...+x7 = 1.

There is a variable y (Cost) = Ax1+Bx2+....+Gx7.

If y > L2 then z = P * (y - R) , 
else if L1 > y > L2 then z = Q * (y - R)  
else z = 0

My obj function is y + z.

How can I formulate this in PuLP?

Thanks,
Raghav
```

1. Conditions on z are ambiguous. Let us reformulate:

Let $0 \le L1 \le L2$. So,

$y \in [0, L1], z = 0$

$y \in [L1, L2], z = Q (y - R)$

$y \in [L2, M], z = P (y - R)$, where $M >> L2$  or $M = L2 + max(A, .., G)$ in the given case.


2. So, we have 3 intervals. Consider a simple case, i.e. forget about x variables for a while.

To model this task we need 3 continuos variables, $y$, and 3 binary varibales, $a$. The code is below.

In [2]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 41.9 MB/s eta 0:00:0000:0100:01


In [5]:
from pulp import *

In [6]:
L1, L2, M = 2, 5, 10
Q, P, R = 3, 2, 1
pw = [1, 2, 3] # pieces
y = LpVariable.dicts("y", pw, lowBound=0, cat=LpContinuous)
a = LpVariable.dicts("a", pw, cat=LpBinary)

In [7]:
m = LpProblem("Problem", LpMaximize)

#### Constraints

In [9]:
# only one piece is in action
m += sum([a[i] for i in pw]) == 1

In [10]:
# 1. z = 0, nothing goes to objective function
m += y[1] <= a[1] * L1

In [11]:
# 2. z = Q*(y - R), therefore  (Q * y[2] - a[2] * R * Q) goes to objective function
m += y[2] >= a[2] * L1
m += y[2] <= a[2] * L2

In [12]:
# 3. z = P * (y - R), therefore  (P * y[3] - a[3] * P * R) goes to objective function
m += y[3] >= a[3] * L2
m += y[3] <= a[3] * M # we need M to make y[3] = 0 when a[1] or a[2] != 0

#### Objective Function

In [13]:
m += y[1] + (y[2] + Q * y[2] - a[2] * R * Q) + (y[3] + P * y[3] - a[3] * P * R)

In [14]:
m

Problem:
MAXIMIZE
-3*a_2 + -2*a_3 + 1*y_1 + 4*y_2 + 3*y_3 + 0
SUBJECT TO
_C1: a_1 + a_2 + a_3 = 1

_C2: a_1 + a_2 + a_3 = 1

_C3: - 2 a_1 + y_1 <= 0

_C4: - 2 a_2 + y_2 >= 0

_C5: - 5 a_2 + y_2 <= 0

_C6: - 5 a_3 + y_3 >= 0

_C7: - 10 a_3 + y_3 <= 0

VARIABLES
0 <= a_1 <= 1 Integer
0 <= a_2 <= 1 Integer
0 <= a_3 <= 1 Integer
y_1 Continuous
y_2 Continuous
y_3 Continuous

In [15]:
# Solution
m.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/b48e484f6bf14c5a98c1ffcacfa81e54-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/b48e484f6bf14c5a98c1ffcacfa81e54-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 40 RHS
At line 48 BOUNDS
At line 52 ENDATA
Problem MODEL has 7 rows, 6 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 28 - 0.00 seconds
Cgl0004I processed model has 5 rows, 5 columns (5 integer (2 of which binary)) and 11 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of -28
Cbc0038I Before mini branch and bound, 2 integers at bound fixed and 3 continuous
Cbc0038I Mini

1

In [11]:
for v in m.variables():
        print(v.name, "=", v.varValue)

a_1 = 0.0
a_2 = 0.0
a_3 = 1.0
y_1 = 0.0
y_2 = 0.0
y_3 = 10.0


3. For the full task we need $3\times7 = 21$ variables, $x$ and 3 binary varibales, $a$.
   Also  we may have $y$ vars to make equations shorter.
   
   Constraints for the second piece (i = 2)
   
```python
x = LpVariable.dicts("x", (pw, range(1, 8)), lowBound=0, cat=LpContinuous)

# ABCDEFG
c = dict([(i, i*0.1) for i in range(1, 8)])

m += sum([x[2][j] for j in range(1, 8)]) == 1 * a[2] # if a2 = 0 all x[2][j] must be = 0

m += y[2] == sum([c[j] * x[2][j] for j in range(1, 8)])

m += y[2] >= a[2] * L1
m += y[2] <= a[2] * L2
```

Now it is simple to write constraints for other two pieces and shorten the code with loops.